# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import plotly.express as px
from sklearn import preprocessing
from sklearn.datasets import load_digits
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import BayesianRidge, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from time import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [ ]:
df.head()

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [ ]:
df.isnull().sum()

In [ ]:
df.shape

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [ ]:
df.info()

In [ ]:
df.hist(bins=35,figsize=(10,20))

In [ ]:
# create Correlation Matrix to understand the correlation between numeric columns
df.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(df.corr(), annot=True)

In [ ]:
df.info()

In [ ]:
num_col = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx','euribor3', 'nr.epmployed']
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact','month', 'day_of_week', 'poutcome', 'y']

In [ ]:
df["age"].hist(bins=10)

In [ ]:
print(df["age"].max())
print(df["age"].min())

In [ ]:
bank_df = df.copy()

In [ ]:
bank_df.info()

In [ ]:
bank_df.describe()

In [ ]:
# Removing the outliers
first_quartile = df['age'].quantile(.25)
third_quartile = df['age'].quantile(.75)
iqr = third_quartile - first_quartile
lower_quartile = first_quartile - 1.5 *iqr
upper_quartile = third_quartile + 1.5 * iqr
bank_df = df.loc[(df['age']> lower_quartile) & (df['age']<upper_quartile)]

In [ ]:
print(bank_df["age"].max())
print(bank_df["age"].min())

In [ ]:
df["duration"].hist(bins=10)

In [ ]:
# Removing outliers from duration
print(bank_df["duration"].max())
print(bank_df["duration"].min())
first_quartile = bank_df['duration'].quantile(.25)
third_quartile = bank_df['duration'].quantile(.75)
iqr = third_quartile - first_quartile
lower_quartile = first_quartile - 1.5 * iqr
upper_quartile = third_quartile + 1.5 * iqr
bank_df = df.loc[(df['duration']> lower_quartile) & (df['duration']< upper_quartile)]

In [ ]:
bank_df["duration"].hist(bins=10)
print(bank_df["duration"].max())
print(bank_df["duration"].min())

In [ ]:
print(bank_df["campaign"].max())
print(bank_df["campaign"].min())

In [ ]:
# Remove outliers from the numerical column - campaign using IQR
first_quartile = bank_df['campaign'].quantile(.25)
third_quartile = bank_df['campaign'].quantile(.75)
iqr = third_quartile - first_quartile
lower_quartile = first_quartile - 1.5 * iqr
upper_quartile = third_quartile + 1.5 * iqr
bank_df = df.loc[(df['campaign']> lower_quartile) & (df['campaign']< upper_quartile)]
bank_df["campaign"].hist(bins=10)

In [ ]:
bank_df.hist(bins=30, figsize=(12, 10))

In [ ]:
# Campaign Acceptance rate based of Number of contact performed
fig, ax = plt.subplots(figsize=(10,5))
sns.countplot(data=bank_df, x='campaign', hue='y')
plt.title("Campaign Acceptance Rate based on Number of Contacts Performed")

In [ ]:
# Campaign Acceptance rate based of Number of contact performed
fig, ax = plt.subplots(figsize=(10,5))
sns.countplot(data=bank_df, x='campaign', hue='y')
plt.title("Campaign Acceptance Rate based on Number of Contacts Performed")

In [ ]:
# Campaign Acceptance rate based of jobs
fig, ax = plt.subplots(figsize=(10,5))
plt.setp(plt.xticks()[1], rotation=45)
sns.countplot(data=bank_df, x='job', hue='y')
plt.title("Campaign Acceptance Rate based on Jobs Performed")

In [ ]:
# Campaign Acceptance rate based of Marital Status
fig, ax = plt.subplots(figsize=(10,5))
plt.setp(plt.xticks()[1], rotation=45)
sns.countplot(data=bank_df, x='marital', hue='y')
plt.title("Campaign Acceptance Rate based on Number of Marital Status")

In [ ]:
# Campaign Acceptance rate based of Education
fig, ax = plt.subplots(figsize=(10,5))
plt.setp(plt.xticks()[1], rotation=45)
sns.countplot(data=bank_df, x='education', hue='y')
plt.title("Campaign Acceptance Rate based on Education")

In [ ]:
# Campaign Acceptance rate based of Education
fig, ax = plt.subplots(figsize=(10,5))
plt.setp(plt.xticks()[1], rotation=45)
sns.countplot(data=bank_df, x='loan', hue='y')
plt.title("Campaign Acceptance Rate based on loan")

In [ ]:
# Campaign Acceptance rate based of Education
fig, ax = plt.subplots(figsize=(10,5))
plt.setp(plt.xticks()[1], rotation=45)
sns.countplot(data=bank_df, x='poutcome', hue='y')
plt.title("Campaign Acceptance Rate based on outcome")

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [ ]:
#Converting Categorical Columns to Numerical Columns using Target Encoder
encoder = preprocessing.LabelEncoder()
encoder_df = bank_df.copy()
def target_encoder(data):
    impute_ordinal = encoder.fit_transform(data)
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

for i in tqdm(range(len(cat_cols))):
    target_encoder(encoder_df[cat_cols[i]])

In [ ]:
encoder_df.describe()


In [ ]:
encoder_df = encoder_df.drop('duration', axis=1) 


In [ ]:
encoder_df.reset_index()

In [ ]:
#standardizing the dataset
scaler = preprocessing.StandardScaler()
std_df = scaler.fit_transform(encoder_df.drop(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact','month', 'day_of_week','campaign', 'previous', 'poutcome', 'emp.var.rate','y'], axis=1))
scaled_df = pd.DataFrame(std_df, columns=['age','pdays','cons.price.idx','cons.conf.idx','euribor3m','nr.employed'])


In [ ]:
scaled_df = pd.DataFrame(std_df, columns=['age','pdays','cons.price.idx','cons.conf.idx','euribor3m','nr.employed'])


In [ ]:
scaled_df['job'] = encoder_df['job'].values
scaled_df['marital'] = encoder_df['marital'].values
scaled_df['education'] = encoder_df['education'].values
scaled_df['default'] = encoder_df['default'].values
scaled_df['housing'] = encoder_df['housing'].values
scaled_df['loan'] = encoder_df['loan'].values
scaled_df['contact'] = encoder_df['contact'].values
scaled_df['month'] = encoder_df['month'].values
scaled_df['day_of_week'] = encoder_df['day_of_week'].values
scaled_df['campaign'] = encoder_df['campaign'].values
scaled_df['previous'] = encoder_df['previous'].values
scaled_df['poutcome'] = encoder_df['poutcome'].values
scaled_df['emp.var.rate'] = encoder_df['emp.var.rate'].values
scaled_df['y'] = encoder_df['y'].values

In [ ]:
scaled_df.head()

In [ ]:
scaled_df.describe()

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
X = scaled_df.drop('y', axis=1)
y = scaled_df["y"].astype('int')
print(X.shape)
print(y.shape)


In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
models = ['Baseline']
train_time = []
train_accuracy = []
test_accuracy = []
accuracy_score = []
AUC_score = []

In [ ]:
# Baseline Model
dummy_clf = DummyClassifier( random_state=42)
start_time = time()
dummy_clf.fit(X_train, y_train)
train_time.append(time() - start_time)
# Training
y_preds = dummy_clf.predict(X_train)
training_score = dummy_clf.score(X_train, y_train)
train_accuracy.append('training_score')
# Test score
y_preds_test = dummy_clf.predict(X_test)
test_score = dummy_clf.score(X_test, y_test)
test_accuracy.append(test_score)
accuracy_score.append('N/A')
AUC_score.append('N/A')


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [ ]:
# Logistics Regression Model 
models.append('Logistics Regression')
log_reg = LogisticRegression(solver='liblinear', random_state=42)
start_time = time()
log_reg.fit(X_train, y_train)
train_time.append(time() - start_time)
y_train_preds = log_reg.predict(X_train)
y_test_preds = log_reg.predict(X_test)


### Problem 9: Score the Model

What is the accuracy of your model?

In [ ]:
train_score = log_reg.score(X_train, y_train)
train_accuracy.append(train_score)
test_accuracy.append(log_reg.score(X_test, y_test))
#accuracy scores
accuracy_score.append(metrics.accuracy_score(y_test, y_pred))
fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
AUC_score.append(metrics.auc(fpr, tpr))


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_test_preds)
print(confusion_matrix)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix)
disp.plot()

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

### RandomForestClassifier

In [ ]:
models.append('RandomForestClassifier')
rand_clf = RandomForestClassifier(random_state=42)
start_time = time()
rand_clf.fit(X_train, y_train)
train_time.append(time() - start_time)
y_pred = rand_clf.predict(X_test)

In [ ]:
# Scoring the model
train_score = rand_clf.score(X_train, y_train)
train_accuracy.append(train_score)
test_accuracy.append(rand_clf.score(X_test, y_test))
#accuracy scores
accuracy_score.append(metrics.accuracy_score(y_test, y_pred))
fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
AUC_score.append(metrics.auc(fpr, tpr))




In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

### KNeighborsClassifier

In [ ]:
models.append('KNeighborsClassifier')
knn = KNeighborsClassifier()
start_time = time()
knn.fit(X_train, y_train)
train_time.append(time() - start_time)
y_pred = knn.predict(X_test)



In [ ]:
# Scoring the model
train_score = knn.score(X_train, y_train)
train_accuracy.append(train_score)
test_accuracy.append(knn.score(X_test, y_test))
#accuracy scores
accuracy_score.append(metrics.accuracy_score(y_test, y_pred))
fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
AUC_score.append(metrics.auc(fpr, tpr))

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

### SVC

In [ ]:
models.append('SVC')
svc = SVC()
start_time = time()
svc.fit(X_train, y_train)
train_time.append(time() - start_time)
y_pred = svc.predict(X_test)

In [ ]:
# Scoring the model
train_score = svc.score(X_train, y_train)
train_accuracy.append(train_score)
test_accuracy.append(svc.score(X_test, y_test))
#accuracy scores
accuracy_score.append(metrics.accuracy_score(y_test, y_pred))
fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
AUC_score.append(metrics.auc(fpr, tpr))

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
model_artifacts = pd.DataFrame({'Model':models, 
                                'Training_Time':train_time, 
                                'Training_Score': train_accuracy, 
                                'Test_Score':test_accuracy,
                                'Accuracy': accuracy_score,
                                'AUC': AUC_score})
model_artifacts

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

<b>Logistics Regression models training, test, AUC scores better than other models. Optimizing Logistics Regression models to improve further.</b>

In [ ]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
log_reg = LogisticRegression()
lgr_cv=GridSearchCV(lgr, grid, cv=10)
lgr_cv.fit(X_train, y_train)

In [ ]:
lgr_cv.best_params_

In [ ]:
#Adjust Performance Metrics
lgr2=LogisticRegression(C=0.1, penalty="l2", solver='liblinear') 
start_time = time()
lgr2.fit(X_train,y_train)
train_time = time() - start_time

In [ ]:
y_pred = lgr2.predict(X_test)

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
print('Precision    : %.3f'%precision_score(y_test, y_pred))
print('Recall       : %.3f'%recall_score(y_test, y_pred))
print('F1 Score     : %.3f'%f1_score(y_test, y_pred))

##### Questions